In [1]:
HOPSWORK_PROJECT_NAME = 'taxi_batch_scoring'  

In [2]:
import os
from dotenv import load_dotenv
from src.paths import PARENT_DIR

# load key-value pairs from .env file located in the parent directory
load_dotenv(PARENT_DIR/'.env')

HOPSWORK_API_KEY = os.environ['HOPSWORK_API_KEY']

In [5]:
from datetime import datetime
import pandas as pd
from src.data import load_raw_data

from_year = 2022
to_year = datetime.now().year
print(f'Downloading raw data {from_year} to {to_year}')

rides = pd.DataFrame()
for year in range(from_year, to_year+1):

    #download data for the whole year
    rides_one_year = load_raw_data(year)

    #append rows
    rides = pd.concat([rides, rides_one_year])

TypeError: unsupported operand type(s) for +: 'datetime.datetime' and 'int'

In [4]:
print(f'{len(rides)=}')

len(rides)=42721352


In [5]:
from src.data import transform_raw_data_into_ts_data

ts_data = transform_raw_data_into_ts_data(rides)

100%|██████████| 265/265 [00:05<00:00, 50.59it/s]


In [6]:
import hopsworks

In [8]:
project = hopsworks.login(
    project=HOPSWORK_PROJECT_NAME,
    api_key_value = HOPSWORK_API_KEY
)

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/27802


In [18]:
feature_store = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.


In [24]:
FEATURE_GROUP_NAME = "time_series_hourly_feature_group"
FEATURE_GROUP_VERSION = 1   

In [25]:
feature_group = feature_store.create_feature_group(
    name=FEATURE_GROUP_NAME,
    version=FEATURE_GROUP_VERSION,
    description="Time-series data at hourly frequency",
    primary_key=['pickup_location_id', 'pickup_hour'],
    event_time='pickup_hour',
)

In [26]:
feature_group.insert(ts_data, write_options={"wait_for_job":False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/27802/fs/27722/fg/30292


Uploading Dataframe: 0.00% |          | Rows 0/2518560 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/27802/jobs/named/time_series_hourly_feature_group_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x20b84a63910>, None)

In [27]:
feature_group.insert(ts_data, write_options={"wait_for_job":False})

Uploading Dataframe: 0.00% |          | Rows 0/2518560 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/27802/jobs/named/time_series_hourly_feature_group_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x20b845f4be0>, None)